In [3]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
# id=str(sys.argv[1])
id='82'
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')


# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)

cursor=cnx.cursor()

In [2]:
loading=pd.read_excel('loading.xlsx')

In [9]:
loading=loading.astype(str)

In [7]:
query='INSERT INTO type_l_r_joist_loading_capacities(caliber,loading_capacity,crossbar_length,camber) values(%s,%s,%s,%s)'

In [10]:
for i in range(len(loading)):
    cursor.execute(query,tuple([loading['calibre'].values[i],
                                loading['capacidad'].values[i],
                                loading['longitud'].values[i],
                                loading['peralte'].values[i],
                                   ]))
cnx.commit()

In [11]:
lr=pd.read_excel('tablas_lr.xlsx')

In [16]:
lr=lr.astype(str)

In [18]:
lr.columns

Index(['largo', 'sku', 'peso', 'm2', 'precio', 'peralte', 'calibre'], dtype='object')

In [24]:
query='INSERT INTO type_l_r_joists(length,sku,weight,m2,price,camber,caliber) values(%s,%s,%s,%s,%s,%s,%s)'

In [26]:
for i in range(len(lr)):
    cursor.execute(query,tuple([lr['largo'].values[i],
                                lr['sku'].values[i],
                                lr['peso'].values[i],
                                lr['m2'].values[i],
                                lr['precio'].values[i],   
                                lr['peralte'].values[i],
                                lr['calibre'].values[i],
                                   ]))
cnx.commit()

In [7]:
data=pd.read_excel('entpint.xlsx',sheet_name='ENTREPAÑO C-26',skiprows=[0,2])

In [10]:
data.columns

Index(['DFONDO M', 'DFRENTE M', 'VFONDO M', 'VLARGO M', 'SKU', 'CALIBRE',
       'KG / M2', 'PESO ', 'M2 ', 'PRECIO VENTA', 'CAPACIDAD DE CARGA (KG)',
       'CON 1 REFUERZO', 'CON 2 REFUERZOS'],
      dtype='object')

In [ ]:
query='INSERT INTO estanteria_entrepanio(development_deep,development_front,deep,length,sku,caliber,kgm2,weight,m2) values(%s,%s,%s,%s,%s,%s,%s)'

In [ ]:
for i in range(len(data)):
    cursor.execute(query,tuple([
        data['DFONDO M'].values[i],
        data['DFRENTE M'].values[i],
        data['VFONDO M'].values[i],
        data['VLARGO M'].values[i],
        data['SKU'].values[i],
        data['CALIBRE'].values[i],
        data['KG / M2'].values[i],
        data['PESO '].values[i],
        data['M2 '].values[i],
        data['CAPACIDAD DE CARGA (KG)'].values[i],
        data['CON 1 REFUERZO'].values[i],
        data['CON 2 REFUERZOS'].values[i],
                                
                                   ]))
cnx.commit()

In [11]:
for i in data.columns:
    print("data['"+i+"'].values[i],")

data['DFONDO M'].values[i],
data['DFRENTE M'].values[i],
data['VFONDO M'].values[i],
data['VLARGO M'].values[i],
data['SKU'].values[i],
data['CALIBRE'].values[i],
data['KG / M2'].values[i],
data['PESO '].values[i],
data['M2 '].values[i],
data['PRECIO VENTA'].values[i],
data['CAPACIDAD DE CARGA (KG)'].values[i],
data['CON 1 REFUERZO'].values[i],
data['CON 2 REFUERZOS'].values[i],
